# Demo of CCC for PSL Meetup

This notebook provides a demonstration of the Cost-of-Capital-Calculator (CCC) for the PSL Meetup on April 29, 2019.

To run this notebook, you will need to follow the instructions to install CCC as described in the CCC README [here](https://github.com/PSLmodels/Cost-of-Capital-Calculator).  In particular, you need to:

* Install the [Anaconda distribution](https://www.anaconda.com/distribution/) of Python
* Download or clone the [CCC repository](https://github.com/PSLmodels/Cost-of-Capital-Calculator) to a directory on your computer
* From the terminal (or Conda command prompt), navigate to the directory to which you cloned this repository and run `conda env create -f environment.yml`
* Then, `source activate ccc-dev` (or `activate ccc-dev` if using a Windows machine)
* Then install by `pip install -e .`

Once you follow the above, you will be ready to work with this Jupyter Notebook.

## First things first, import necessary packages

In [1]:
# import packages
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# import CCC classes that we'll work with
from ccc.data import Assets
from ccc.parameters import Specification
from ccc.calculator import Calculator
# to print bokeh plots inline
output_notebook()

Loading BokehJS ...

## Create an instance of the Assets class

This is the class that contains the data that underlie CCC.  The basic object is a Pandas DataFrame where each row represents a combination of a specific type of asset, industry (approximately 6-digit NAICS) and tax treatment.  The columns represent names and codes for the asset and industry classifications, the tax depreciation rules used for that asset, and the rate of economic depreciation (`delta`).

In [2]:
assets = Assets()
assets.df.head(n=5)

,Unnamed: 0,tax_treat,assets,bea_asset_code,bea_ind_code,Industry,minor_code_alt,major_asset_group,minor_asset_group,major_industry,GDS Class Life,GDS Life,ADS Life,System,Method,asset_name,delta
0,0,corporate,0.0,RD32,110C,Farms,111.0,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",7.0,7.0,10.0,GDS,Expensing,Aerospace products and parts manufacturing,0.22
1,1,corporate,0.0,RD32,113F,"Forestry, fishing, and related activities",113.0,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",7.0,7.0,10.0,GDS,Expensing,Aerospace products and parts manufacturing,0.22
2,2,corporate,0.0,RD32,113F,"Forestry, fishing, and related activities",114.0,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",7.0,7.0,10.0,GDS,Expensing,Aerospace products and parts manufacturing,0.22
3,3,corporate,0.0,RD32,2110,Oil and gas extraction,211110.0,Intellectual Property,Intellectual Property,Mining,7.0,7.0,10.0,GDS,Expensing,Aerospace products and parts manufacturing,0.22
4,4,corporate,0.0,RD32,2120,"Mining, except oil and gas",212110.0,Intellectual Property,Intellectual Property,Mining,7.0,7.0,10.0,GDS,Expensing,Aerospace products and parts manufacturing,0.22


## Create an instance of the Specification class

This class contains the parameters.  It has methods that load the data from a file that contains the default parameter values (`default_parameters.json`) and then stores them as attributes of the `Specification` class object.  

`p.u` looks into the instance of this class named `p` and executing the cell below will show you that `p.u` is dictionary containtin marginal tax rates on corporate and pass-through income.

Note that the `Specification` class has some arguments, such as `call_tc`, which defaults to `False`, but if set to `True` will call the Tax-Calculator to estimate marginal tax rates on individual filers' income.

In [3]:
# Create an instance of the Specification class
# p = Specification()
p = Specification(call_tc=True)
# Look at attributes
p.u

year:  2019
{'tau_nc': array([0.19562461]), 'tau_div': array([0.18968229]), 'tau_int': array([0.3138236]), 'tau_scg': array([0.29351499]), 'tau_lcg': array([0.19041333]), 'tau_td': array([0.21971122]), 'tau_h': array([0.04452513])}


{'c': 0.21, 'nc': array([0.19562461])}

## Create an instance of the Calculator class

This class does the calculations on the data.  It takes as arguments the data object (named `assets` here) and the parameters object (named `p` here).

Excuting the cell below creates an instance of the calculator class with these data and parameters, but does not yet excecute any calculations.

In [4]:
# Create an instance of the Calculator class
calc1 = Calculator(p, assets)

With an instance of the `Calculator` class created, we can start doing some calculations with these data.

For instance, we can compute a table of the share of corporate vs non-corporate assets across each industry.    

In [5]:
# Look at shares of assets across industry
calc1.asset_share_table()

,Industry,Corporate,Pass-Through
0,"Agriculture, forestry, fishing, and hunting",0.309688,0.690312
1,Mining,0.826839,0.173161
2,Utilities,0.948540,0.051460
3,Construction,0.286840,0.713160
4,Manufacturing,0.859014,0.140986
5,Wholesale trade,0.715166,0.284834
6,Retail trade,0.728628,0.271372
7,Transportation and warehousing,0.789652,0.210348
8,Information,0.918722,0.081278
9,Finance and insurance,0.786056,0.213944


## Specifying a reform policy

To see some more interesting results, we will want to create another `Calculator` object with a change in policy or economic assumptions.  We can do this in a way analogous to our original instantiation of the `Calculator` object above.  In particular, we'll need to create a new `Specification` object (we can use the same underlying data, which was in the object we named `assets`).

In the code below, we'll specify our "reform" as current law tax policy for 2026 (the baseline parameters above came from the default model year, 2019).  In addition, we'll increase the corporate income tax rate from 21 to 25%.

In [9]:
# Create another policy
p2 = Specification(year=2026)
p2.update_specification({'CIT_rate': {2026: 0.35}})
calc2 = Calculator(p2, assets)

## Tabular output

Now with two `Calculator` objects named `calc1` and `calc2` (representing the baseline and reform policies), we are ready to compute some of the changes in effective tax rates, cost of capital, or other variables measured in this model.

We start with an overall summary table showing the marginal effective total tax rates (METTRs) for all investments, corporate investments, and pass-through investments under varying financing assumptions.  This is done through the `summary_table` function.  It takes a calculator object as an argument.

In [10]:
# Look at differences in METTRs between the two policies
calc1.summary_table(calc2) # calc1 is the baseline, calc2 the reform

,,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,18.228124,26.010085,7.781961
1,Corporations,18.998045,29.247574,10.249529
2,Equity Financed,21.935332,35.909289,13.973957
3,Debt Financed,7.011359,-8.518323,-15.529682
4,Pass-Through Entities,18.712599,23.086660,4.374061
5,Equity Financed,16.632096,23.061862,6.429765
6,Debt Financed,27.276515,23.409483,-3.867032


the `Calculator.summary_table()` method defaults to showing the results for the METTR, but there is a keyword argument that would allow you to view the output for other variables computed in CCC.  We can use this to see changes in the cost of capital (denoted by $\rho$ in the model):

In [11]:
# Look at how cost of capital changed
calc1.summary_table(calc2, output_variable='rho')

,,Cost of Capital Under Baseline Policy,Cost of Capital Under Reform Policy,Change from Baseline (pp)
0,Overall,5.506884,6.086076,0.579192
1,Corporations,5.559227,6.364562,0.805336
2,Equity Financed,6.579346,8.013868,1.434521
3,Debt Financed,3.395925,2.909946,-0.485979
4,Pass-Through Entities,5.444491,5.754119,0.309628
5,Equity Financed,6.160818,6.675681,0.514863
6,Debt Financed,3.693834,3.507334,-0.186501


One can also save results to disk by specifying an output type ('excel', 'json', 'csv', 'tex') and a file path: 

In [12]:
# Save these results to disk
calc1.summary_table(calc2, output_variable='rho', output_type='excel', path='cc_table.xlsx')

There are also `Calculator` methods to compute summary tables by asset type or industry.  These are computed in the next two cells.

In [13]:
# Summary by asset type
calc1.asset_summary_table(calc2)

,Category,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,18.228124,26.010085,7.781961
1,Corporate,18.998045,29.247574,10.249529
2,Equipment,8.007243,23.838921,15.831678
3,Structures,19.691960,28.853385,9.161424
4,Intellectual Property,8.007243,11.798031,3.790788
5,Inventories,30.656094,41.218548,10.562455
6,Land,27.325722,36.236418,8.910696
7,Pass-through,18.712599,23.086660,4.374061
8,Equipment,2.451141,15.122501,12.671360
9,Structures,17.539964,21.438864,3.898900


In [14]:
# Summary by industry
calc1.industry_summary_table(calc2)

,Category,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,18.228124,26.010085,7.781961
1,Corporate,18.998045,29.247574,10.249529
2,"Agriculture, forestry, fishing, and hunting",19.857835,31.833009,11.975175
3,Mining,10.976008,19.014814,8.038807
4,Utilities,9.029427,21.159254,12.129827
5,Construction,20.387255,31.578546,11.191290
6,Manufacturing,19.342030,28.142251,8.800222
7,Wholesale trade,24.829453,35.661290,10.831837
8,Retail trade,25.735453,35.892062,10.156610
9,Transportation and warehousing,12.253130,21.726453,9.473323


## Visualizations

If one wants to visualize the effects of changes in tax policy, the `Calculator` class has a few methods for this.  

We can use the `grouped_bar` method to show differential effects across assets (the default):

In [15]:
# Visualizing changes by asset
aplot = calc1.grouped_bar(calc2)
show(aplot)

or we can change from the default asset category split to a split by industry by changing the value of the `group_by_asset` keyword argument:

In [16]:
# Visualizing changes by industry
iplot = calc1.grouped_bar(calc2, group_by_asset=False)
show(iplot)

There's also a plot that illustrates the range of the effects of taxes on investments across asset types, by showing the min, max, and mean values:

In [17]:
# Plot to show variation in METTRs across assets
rplot = calc1.range_plot(calc2, output_variable='metr')
show(rplot)

changing to show the effects on pass-through businesses and for a different output variable:

In [18]:
# Plot to show variation in METTRs across assets
rplot2 = calc1.range_plot(calc2, corporate=False, output_variable='mettr')
show(rplot2)

There's a bubble plot too, though it doesn't yet allow for many options (currently only plots METTRs for corporate entities):

In [19]:
bplot = calc1.asset_bubble(calc2)
show(bplot)

## Summary

This notebook provides a brief example of how one would work with CCC.  Please explore the source code to see additional flexiblity in the functions.  And please leave any questions or suggestions in the CCC repo at [https://github.com/PSLmodels/Cost-of-Capital-Calculator/issues](https://github.com/PSLmodels/Cost-of-Capital-Calculator/issues).